# Pre-Process Data: Specific DSAID

In [12]:
"""Pre-Process Data

Convert the raw data counts into sc-RNAseq compatible data format.

Structure:
    1. Imports, Variables, Functions
    2. Load Data
    3. Convert to `adata` object

"""

# 1. Imports, Variables, Functions
# imports
import numpy as np, os, sys, pandas as pd, scanpy as sc
import anndata as ad

# variables
example_data_path = (
    "/aloy/home/ddalton/projects/disease_signatures/data/DiSignAtlas/tmp/DSA00015.csv"
)

# functions


# 2. Load Data
df = pd.read_csv(example_data_path, index_col=0)

# 3. Convert to `adata` object


# Extract cell identifiers and gene expression data
cell_ids = df.iloc[:, 0]
gene_expression_data = df.iloc[:, 1:].values
gene_names = df.columns[1:]


# Create an AnnData object
adata = ad.AnnData(X=gene_expression_data)


# Add cell and gene metadata
adata.obs["cell_ids"] = cell_ids.values
adata.var["gene_symbols"] = gene_names

# Preprocess Data: Specific Diseases

In [1]:
"""Pre-Process Data

Convert the raw data counts into sc-RNAseq compatible data format.

Structure:
    1. Imports, Variables, Functions
    2. Load Data
    3. Convert to `adata` object

"""

# 1. Imports, Variables, Functions
# imports
import numpy as np, os, sys, pandas as pd, scanpy as sc
import anndata as ad
import logging
from tqdm import tqdm

logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(message)s")
from matplotlib import pyplot as plt
from collections import Counter

# import random
import random

from sklearn.model_selection import train_test_split


# variables
diseases_of_interest_set = {"Influenza", "Colorectal Carcinoma", "Breast Cancer"}
diseases_of_interest_set = {
    "Crohn's Disease",
    "Ulcerative Colitis",
    "Lung Cancer",
    "Lung Adenocarcinoma",
    "Breast Cancer",
    "Psoriasis",
}


example_data_path = (
    "/aloy/home/ddalton/projects/disease_signatures/data/DiSignAtlas/tmp/DSA00123.csv"
)

df_info_path = os.path.join(
    "/aloy",
    "home",
    "ddalton",
    "projects",
    "disease_signatures",
    "data",
    "DiSignAtlas",
    "Disease_information_Datasets_extended.csv",
)


large_df_path = "/aloy/home/ddalton/projects/disease_signatures/data/DiSignAtlas/DiSignAtlas.exp_prof_merged.csv"


# functions
def get_exp_prof(dsaids_interest):
    """Get Expression Profiles"""

    # variables
    file_dir = "/aloy/home/ddalton/projects/disease_signatures/data/DiSignAtlas/tmp/"
    first = True
    for dsaid in tqdm(dsaids_interest):
        __df = pd.read_csv(os.path.join(file_dir, f"{dsaid}.csv"))
        if first:
            df_global = __df
            first = False
        else:
            df_global = pd.concat([df_global, __df], axis=0)
    return df_global


def get_dataset_info(ids, df_info):
    """Get Disease Information"""
    dsaid_2_accession = dict(zip(df_info["dsaid"], df_info["accession"]))

    dataset_accessions = [dsaid_2_accession[id.split(";")[0]] for id in ids]

    accession_2_id = {k: v for v, k in enumerate(set(dataset_accessions))}
    dataset_ids = [accession_2_id[accession] for accession in dataset_accessions]

    return dataset_accessions, dataset_ids


def get_disease_info(ids, df_info):
    """Get Disease Information"""
    dsaid_2_disease = dict(zip(df_info["dsaid"], df_info["disease"]))
    disease_types = list()
    for id in ids:
        if id.split(";")[2] == "Control":
            disease_types.append("Control")
        elif id.split(";")[2] == "Case":
            disease_types.append(dsaid_2_disease[id.split(";")[0]])
        else:
            print(f"Error with ID: {id}")

    map_2_id = {k: v for v, k in enumerate(set(disease_types))}

    disease_ids = [map_2_id[x] for x in disease_types]

    return disease_types, disease_ids


# 2. Load Data
# df = pd.read_csv(example_data_path)

df_info = pd.read_csv(df_info_path)


# Query data to retrieve dsaids of interest
library_strategies_of_interest_set = {"RNA-seq", "Microarray"}
QUERY = "disease in @diseases_of_interest_set & library_strategy in @library_strategies_of_interest_set & organism == 'Homo sapiens'"
dsaids_interest = df_info.query(QUERY)["dsaid"].to_list()
logging.info(f"Nº of DSAIDs of interest: {len(dsaids_interest)}")

# skip_rows_idxs = get_skip_rows(dsaids_interest)

df = get_exp_prof(dsaids_interest)

# 3. Convert to `adata` object
# load dataframe
# df = pd.read_csv(large_df_path, skiprows=skip_rows_idxs, index_col=0)
logging.info(f"Loaded dataframe with shape: {df.shape}")

# drop non significant rows
# Calculate the number of NaNs in each row
nan_counts = df.isna().sum(axis=1)

# Filter the DataFrame to keep only rows with NaNs less than or equal to 18,000
df = df[nan_counts <= 18000]
logging.info(f"Filtered dataframe with shape: {df.shape}")

# Filter out Unknown samples
mask = [False if id.split(";")[2] == "Unknown" else True for id in df.iloc[:, 0].values]

df = df[mask]

logging.info(f"Filtered out Unkowns from dataframe with shape: {df.shape}")


# 3. Convert to `adata` object
# Extract cell identifiers and gene expression data
ids = df.iloc[:, 0]
gene_expression_data = df.iloc[:, 1:].values
gene_names = df.columns[1:]

# Create an AnnData object
adata = ad.AnnData(X=gene_expression_data)


# Add cell and gene metadata
adata.obs["ids"] = ids.values
adata.var["gene_symbols"] = gene_names
adata.var["index"] = gene_names
adata.var["gene_name"] = gene_names

NameError: name 'df_info' is not defined

In [ ]:
# generate celltype and celltype_id columns in adata.obs
disease_types, disease_ids = get_disease_info(ids.values, df_info=df_info)
adata.obs["celltype"] = disease_types
adata.obs["celltype_id"] = disease_ids

logging.info(f"Counts of each disease type: {Counter(disease_types)}")

2024-07-20 16:41:24,867 - Counts of each disease type: Counter({'Control': 1764, 'Breast Cancer': 1740, 'Colorectal Carcinoma': 1115, 'Influenza': 471})


In [ ]:
# generate dataset ids column in adata.obs
dataset_ids, batch_ids = get_dataset_info(ids.values, df_info=df_info)

adata.obs["dataset_id"] = dataset_ids
adata.obs["batch_id"] = batch_ids

logging.info(f"Counts of each dataset id: {Counter(dataset_ids)}")

2024-07-21 09:17:33,808 - Counts of each dataset id: Counter({'GSE54002': 431, 'GSE87211': 361, 'GSE41258': 321, 'GSE55276': 234, 'GSE55277': 234, 'GSE5364': 194, 'GSE45827': 162, 'GSE101702': 157, 'GSE27562': 146, 'GSE24124': 135, 'GSE47756': 127, 'GSE16443': 119, 'GSE42568': 119, 'GSE164191': 119, 'GSE29044': 107, 'GSE37751': 106, 'GSE33113': 94, 'GSE123086': 92, 'GSE9348': 80, 'GSE62932': 79, 'GSE29366': 67, 'GSE29431': 64, 'GSE61304': 58, 'GSE74604': 58, 'GSE10810': 56, 'GSE110933': 48, 'GSE34205': 48, 'GSE36295': 48, 'GSE24514': 47, 'GSE3744': 45, 'GSE139038': 45, 'GSE20437': 40, 'GSE100179': 38, 'GSE100150': 37, 'GSE32488': 37, 'GSE123087': 36, 'GSE3964': 35, 'GSE64392': 35, 'GSE32323': 34, 'GSE22598': 34, 'GSE77953': 30, 'GSE31471': 30, 'GSE80759': 30, 'GSE122183': 30, 'GSE10715': 30, 'GSE9574': 29, 'GSE41011': 29, 'GSE70468': 28, 'GSE14297': 25, 'GSE4183': 23, 'GSE4107': 22, 'GSE27131': 21, 'E-MTAB-5385': 21, 'GSE20266': 20, 'GSE11545': 20, 'GSE90708': 19, 'GSE25487': 19, 'GSE7

In [ ]:
# We combine both the disease type with the dataset as to
# shuffle even more the data - minimize bias in train/test split
labels = np.array([a + b for a, b in zip(disease_types, dataset_ids)])

# Generate indices for the data points
indices = np.arange(len(labels))

"""train_test_split cannot handle single label indexes

Because of this we will manually deal with these cases!

"""
indices_single_label = [i for i, x in enumerate(labels) if list(labels).count(x) == 1]
labels_single_label = labels[indices_single_label]

logging.info(f"Nº of single label indexes: {len(indices_single_label)}")


remaining_indices = [i for i in indices if i not in indices_single_label]
remaining_labels = labels[remaining_indices]

# Perform stratified split on the remaining indices
train_indices, test_indices = train_test_split(
    remaining_indices, test_size=0.2, stratify=remaining_labels, random_state=42
)

for idx in indices_single_label:

    if random.random() < 0.2:
        test_indices.append(idx)
    else:
        train_indices.append(idx)


test_indices.sort()
train_indices.sort()

batch_ids = np.zeros(len(labels))
batch_ids[test_indices] = 1


logging.info(f"Nº of train samples: {np.sum(~batch_ids.astype(bool))}")
logging.info(f"Nº of test samples: {np.sum(batch_ids.astype(bool))}")


adata.obs["train_test"] = batch_ids
adata.obs["str_batch"] = batch_ids.astype(int).astype(str)

2024-07-20 17:19:19,651 - Nº of single label indexes: 2
2024-07-20 17:19:19,656 - Nº of train samples: 4072
2024-07-20 17:19:19,657 - Nº of test samples: 1018


In [ ]:
# save adata object
# adata.write("../data/test_1/test_1.h5ad")
adata.write("../data/test_1/test_2.h5ad")

... storing 'str_batch' as categorical


In [ ]:
adata.obs

,ids,celltype,celltype_id,dataset_id,batch_id,train_test,batch_str,str_batch
0,DSA00057;GSM2062351;Control,Control,0,GSE77953,23,0.0,0.0,0.0
1,DSA00057;GSM2062352;Control,Control,0,GSE77953,23,1.0,1.0,1.0
2,DSA00057;GSM2062353;Control,Control,0,GSE77953,23,0.0,0.0,0.0
3,DSA00057;GSM2062354;Control,Control,0,GSE77953,23,0.0,0.0,0.0
4,DSA00057;GSM2062355;Control,Control,0,GSE77953,23,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...
5085,DSA10295;GSM1012438;Case,Colorectal Carcinoma,3,GSE41258,16,0.0,0.0,0.0
5086,DSA10295;GSM1012444;Case,Colorectal Carcinoma,3,GSE41258,16,1.0,1.0,1.0
5087,DSA10295;GSM1012448;Case,Colorectal Carcinoma,3,GSE41258,16,0.0,0.0,0.0
5088,DSA10295;GSM1012449;Case,Colorectal Carcinoma,3,GSE41258,16,0.0,0.0,0.0


In [ ]:
# we will create an "artifical" cell type which
# will represent the diseases